# Hagrid, Provisioning and Deployment

The goal for `hagrid` is to be a friendly helper in navigating the world of deployment and provisioning the `syft` stack and to take a multitude of complex steps and turn them into a handful of uniform and consistent commands.

`HAGrid` can deploy to a large variety of targets and by leveraging other tools and formats this can be extended. More specifically, `hagrid` can deploy to:
- `docker`
- any ip (`x.x.x.x`)
- `localhost`
- `azure`
- `gcp`
- `aws` (todo)
- `kubernetes` (coming soon)

## Launch to `Docker`

For some commands we can pass `--cmd=true` and `hagrid` will print a dry run of what it was going to do. This is a handy way to understand what `hagrid` is really doing.

In [1]:
!hagrid launch test_domain domain to docker:8081 --tag=0.7.0 --dev --cmd

✅ Updated HAGrid from branch: 0.7.0m
⠋ Updating HAGrid from branch: 0.7.0
⠇ Checking for Docker Service   ice   
✅ Docker service is running
✅ Git 2.34.1
✅ Docker 23.0.0
✅ Docker Compose 2.15.1


 _   _       _     _                 _   _                       _
| | | |     | |   | |               | | | |                     | |
| |_| | ___ | | __| |   ___  _ __   | |_| | __ _ _ __ _ __ _   _| |
|  _  |/ _ \| |/ _` |  / _ \| '_ \  |  _  |/ _` | '__| '__| | | | |
| | | | (_) | | (_| | | (_) | | | | | | | | (_| | |  | |  | |_| |_|
\_| |_/\___/|_|\__,_|  \___/|_| |_| \_| |_/\__,_|_|  |_|   \__, (_)
                                                            __/ |
                                                           |___/
        
Launching a PyGrid Domain node on port 8081!

  - NAME: test_domain
  - RELEASE: development
  - ARCH: linux/amd64
  - TYPE: domain
  - DOCKER_TAG: 0.7.0-dev
  - GIT_HASH: 503940bb16bc18585864355a0ab3b90e0cb3a15d
  - HAGRID_VERSION: 0.2.126
  - PORT: 8081
 